In [1]:
from iGTP_Linear import *
from iGTP_model_modified_scvi_sample_final_update import *
from preprocess import *
from learning_utilies import *
import os 
import scanpy as sc

Global seed set to 0
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
ppi_file_list=os.listdir('/home/iGTP/ppi/')
ppi_file_list=[i for i in ppi_file_list if'.ipynb_checkpoints' not in i]
ppi_file_list.sort()
print(ppi_file_list)
tp_file_list=os.listdir('/home/iGTP/TP/')
tp_file_list=[i for i in tp_file_list if'.ipynb_checkpoints' not in i]
tp_file_list=[i for i in tp_file_list if 'brain' in i or 'GO' in i]
tp_file_list.sort()
print(tp_file_list)
data_file_list=os.listdir('/home/iGTP/iGTP_data')
data_file_list=[i for i in data_file_list if'._' not in i]
data_file_list=[i for i in data_file_list if'.h5ad' in i]
data_file_list.sort()
print(data_file_list)

['PC.txt', 'PC_homo_v11.txt', 'PPI_compiled.txt']
['GO_merged.gmt', 'brain_cell_type_function_AD_pathways.gmt']
['Adamson2016.h5ad', 'GBM.h5ad', 'Microglia_ipsc_only_2022.h5ad', 'NormanWeissman2019_filtered.h5ad', 'ShifrutMarson2018.h5ad', 'TianKampmann2019_day7neuron.h5ad', 'covid19.h5ad', 'covid_pbmc_ifn_test.h5ad', 'covid_pbmc_ifn_train.h5ad', 'covid_pbmc_ifn_train_test.h5ad', 'pbmc3k.h5ad', 'pbmc_IFN.h5ad', 'pbmc_IFN_1.h5ad', 'pbmc_IFN_filtered.h5ad', 'pbmc_IFN_filtered_1.h5ad', 'resilience.h5ad', 'resilience_ips.h5ad']


In [3]:
data_file_list[13]

'pbmc_IFN_filtered.h5ad'

In [4]:
args={}
args['tp_overlap_fraction']=0.1
args['encoder_layer_list']=[1500]
args['beta']=0.00005
args['encoder_normal']='batch'
args['init_type']='pos_uniform'
args['recon_loss']='mse'
args['drop_out']=0.2
args['eps']=0.001
args['cv_fold']=5
args['learning_rate']=0.001
args['learning_rate_weight_decay']=0.000054
args['n_epochs']=500
args['train_patience']=10
args['test_patience']=10
args['data_num_workers']=1
args['save_fold']=True
args['save_best']=True
args['anneal_start']=None
args['anneal_time']=None
args['sc_data_path']='/home/iGTP/iGTP_data/'+data_file_list[13]
args['tp_file_path']='/home/iGTP/TP/'+tp_file_list[0]
args['ppi_file_path']='/home/iGTP/ppi/'+ppi_file_list[1]
args['model_dir']='./save_model/'+data_file_list[13].replace('.h5ad','')+'/'
args['vb_nu']=15000
args['model_prefix']=tp_file_list[0].replace('.gmt','')+'_'+ppi_file_list[1].replace('.txt','')+args['recon_loss']+'_'+args['init_type']+'_'+str(args['vb_nu'])+'batch'
args['batch_size']=4000
args['drop_last_batch']=False
args['device_nu']=1
args['using_gpu']=True
args['z_sample']=1
adata = sc.read(args['sc_data_path'])
print(adata.X.shape)

(13672, 35635)


In [5]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-') 
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.n_genes_by_counts < 10000, :]
adata = adata[adata.obs.pct_counts_mt < 20, :]
adata = adata[adata.obs.total_counts < 50000, :]
sc.pp.normalize_total(adata, target_sum=1e5)
sc.pp.log1p(adata)
#sc.pp.highly_variable_genes(adata, n_top_genes=args['vb_nu'])
print(adata.X.shape)
#adata.raw = adata
#adata = adata[:, adata.var.highly_variable]
print(adata.X.shape)

/usr/local/lib/python3.8/dist-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


(13672, 14949)
(13672, 14949)


In [6]:
adata.raw.X

<13672x35635 sparse matrix of type '<class 'numpy.float64'>'
	with 9530688 stored elements in Compressed Sparse Row format>

In [7]:
if not os.path.exists(args['model_dir']):
            # Create the directory if it doesn't exist
    os.mkdir(args['model_dir'])
    print("Directory created successfully.")
else:
    print("Directory already exists.")
    print(args['model_dir'])

Directory already exists.
./save_model/pbmc_IFN_filtered/


In [8]:
data_x_s,mask_list_dict,args,seq_gene_df,seq_gene_df_su, TP_df, ppi_df_1,data=pre_process(args,adata)

10370
Number of Gene 10119 in PPI
Create TP->Gene mask
Number of TP 1296, and Gene 10370
Create Gene->Gene mask
Number of Gene 10370, and Gene 10370
The number of gene in TP Yes was 10370.00 


In [9]:
all_data_df={}
all_data_df['data_x_s']=data_x_s
all_data_df['mask_dict']=mask_list_dict
all_data_df['args']=args
all_data_df['seq_gene_df']=seq_gene_df
all_data_df['seq_gene_df_su']=seq_gene_df_su
all_data_df['TP_df']=TP_df
all_data_df['ppi_df_1']=ppi_df_1
all_data_df['data']=data
args['model_data_dir']=args['model_dir']+'processed_data/'
if not os.path.exists(args['model_data_dir']):
            # Create the directory if it doesn't exist
    os.mkdir(args['model_data_dir'])
    print("Directory created successfully.")
else:
    print("Directory already exists.")
model_data_file=args['model_data_dir']+args['model_prefix']+'.pkl'
with open(model_data_file, 'wb') as file:
    pickle.dump(all_data_df, file)

Directory already exists.


In [ ]:
args

In [ ]:
kfold=KFoldTorch(args) 
kfold.train_kfold(iGTP,mask_list_dict,data_x_s)